In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Punto 8**

In [13]:
train = pd.read_csv("../data/raw/train.csv")          # train con Survived real
test_modelo  = pd.read_csv("../data/raw/test_modelo.csv")    # predicciones con Random Forest


In [19]:
columnas_numericas = ["Age", "Fare", "Pclass", "SibSp", "Parch"]

for df in (train, test_modelo):

    for col in columnas_numericas:
        if col in df.columns:
            df.loc[:, col] = pd.to_numeric(df[col], errors="coerce")

    
    if "PassengerId" in df.columns:
        df.loc[:, "PassengerId"] = df["PassengerId"].astype(str)

if "Survived" not in test_modelo.columns and "Random Forest" in test_modelo.columns:
    test_modelo = test_modelo.copy()
    rf_vals = pd.to_numeric(test_modelo["Random Forest"], errors="coerce")
    rf_bin = np.rint(rf_vals).clip(0, 1)  
    test_modelo["Survived"] = rf_bin.astype("Int64")  

def calcular_edad_promedio(df):
    """Calcula la edad promedio de los pasajeros (NaN si no existe Age)."""
    return df["Age"].mean() if "Age" in df.columns else np.nan

def contar_sobrevivientes_y_fallecidos(df):
    """Cuenta cuantos pasajeros sobrevivieron y cuantos fallecieron.
       Si no hay 'Survived', retorna ('N/A','N/A')."""
    if "Survived" not in df.columns:
        return "N/A", "N/A"

    s = pd.to_numeric(df["Survived"], errors="coerce").dropna()
    if s.empty:
        return 0, 0
    s_bin = np.rint(s).clip(0, 1).astype(int)
    sobrevivientes = int((s_bin == 1).sum())
    fallecidos = int((s_bin == 0).sum())
    return sobrevivientes, fallecidos

def calcular_tarifa_promedio_primera_clase(df):
    """Tarifa promedio para Pclass == 1 (NaN si no existen columnas)."""
    if "Pclass" not in df.columns or "Fare" not in df.columns:
        return np.nan
    return df.loc[df["Pclass"] == 1, "Fare"].mean()

def contar_pasajeros_con_familia(df):
    """Cuenta pasajeros con al menos un familiar (SibSp + Parch > 0)."""
    if "SibSp" not in df.columns or "Parch" not in df.columns:
        return 0
    total_fam = df["SibSp"].fillna(0) + df["Parch"].fillna(0)
    return int((total_fam > 0).sum())

def obtener_edades_extremas(df):
    """Devuelve (edad_min, edad_max). NaN si no existe Age."""
    if "Age" not in df.columns:
        return np.nan, np.nan
    return df["Age"].min(), df["Age"].max()

def contar_pasajeros_por_puerto_embarque(df):
    """Cuenta pasajeros por puerto (incluye NaN como categoría)."""
    if "Embarked" not in df.columns:
        return pd.Series(dtype=int)
    return df["Embarked"].value_counts(dropna=False).astype(int)

def comparar_solos_vs_con_familia(df):
    """Devuelve Serie con índices 'Solo' y 'Con_Familia'."""
    if "SibSp" not in df.columns or "Parch" not in df.columns:
        return pd.Series({"Solo": 0, "Con_Familia": 0}, dtype=int)
    total_fam = df["SibSp"].fillna(0) + df["Parch"].fillna(0)
    solo = int((total_fam == 0).sum())
    con_fam = int((total_fam > 0).sum())
    return pd.Series({"Solo": solo, "Con_Familia": con_fam}, dtype=int)


datasets_a_analizar = {
    "train": train,
    "test_modelo": test_modelo
}

filas_resumen = []
detalles_embarque = {}
detalles_familia = {}

for nombre, df in datasets_a_analizar.items():
    edad_prom = calcular_edad_promedio(df)
    sobrevivientes, fallecidos = contar_sobrevivientes_y_fallecidos(df)
    tarifa_primera = calcular_tarifa_promedio_primera_clase(df)
    con_familia = contar_pasajeros_con_familia(df)
    edad_min, edad_max = obtener_edades_extremas(df)
    conteo_embarque = contar_pasajeros_por_puerto_embarque(df)
    conteo_familia = comparar_solos_vs_con_familia(df)

    detalles_embarque[nombre] = conteo_embarque
    detalles_familia[nombre] = conteo_familia

    filas_resumen.append({
        "Dataset": nombre,
        "(a) Edad Promedio": edad_prom,
        "(b) Sobrevivieron": sobrevivientes,
        "(b) Fallecieron": fallecidos,
        "(c) Tarifa Promedio 1ra Clase": tarifa_primera,
        "(d) Pasajeros con Familia": con_familia,
        "(e) Edad Minima": edad_min,
        "(e) Edad Maxima": edad_max,
        "(g) Viajaban Solos": int(conteo_familia.get("Solo", 0)),
        "(g) Viajaban con Familia": int(conteo_familia.get("Con_Familia", 0)),
    })

resumen_final = pd.DataFrame(filas_resumen)

orden = [
    "Dataset",
    "(a) Edad Promedio",
    "(e) Edad Minima",
    "(e) Edad Maxima",
    "(c) Tarifa Promedio 1ra Clase",
    "(d) Pasajeros con Familia",
    "(g) Viajaban Solos",
    "(g) Viajaban con Familia",
    "(b) Sobrevivieron",
    "(b) Fallecieron",
]
resumen_final = resumen_final.reindex(columns=orden)

for c in ["(a) Edad Promedio", "(e) Edad Minima", "(e) Edad Maxima", "(c) Tarifa Promedio 1ra Clase"]:
    if c in resumen_final.columns:
        resumen_final[c] = resumen_final[c].astype(float).round(2)

print("Resumen estadístico completo:")
display(resumen_final)

Resumen estadístico completo:


,Dataset,(a) Edad Promedio,(e) Edad Minima,(e) Edad Maxima,(c) Tarifa Promedio 1ra Clase,(d) Pasajeros con Familia,(g) Viajaban Solos,(g) Viajaban con Familia,(b) Sobrevivieron,(b) Fallecieron
0,train,29.70,0.42,80.0,84.15,354,537,354,342,549
1,test_modelo,30.27,0.17,76.0,94.28,165,255,165,111,307


**(a) Edad promedio de los pasajeros**
La edad promedio de los pasajeros del Titanic es aproximadamente 29.7 años en el conjunto de entrenamiento y 30.27 años en el conjunto de prueba.
Esto indica que, en general, los pasajeros eran adultos jóvenes, con una distribución de edades similar entre ambos conjuntos de datos.

**(b) Pasajeros que sobrevivieron y fallecieron**

En el conjunto train, 342 pasajeros sobrevivieron mientras que 549 fallecieron.
En el conjunto test_modelo, que contiene las predicciones del modelo Random Forest, 111 fueron clasificados como sobrevivientes y 307 como fallecidos.
Esto muestra que, tanto en los datos reales como en las predicciones, el número de fallecidos supera notablemente al de sobrevivientes.

**(c) Tarifa promedio de los pasajeros de primera clase**

Los pasajeros de primera clase pagaron una tarifa promedio de 84.15 en el conjunto train y 94.28 en test_modelo.
Esto sugiere que los precios variaban entre los conjuntos, probablemente por diferencias en el tamaño de muestra o por los pasajeros incluidos en cada uno.

**(d) Pasajeros que viajaron con familiares**

En el conjunto de entrenamiento (train), 354 pasajeros viajaban con al menos un familiar a bordo, mientras que 537 lo hacían solos.
En el conjunto de prueba (test_modelo), 165 pasajeros viajaban acompañados por familiares, frente a 255 que viajaban solos.

**(e) Edad más joven y más vieja**

La persona más joven registrada tenía 0.42 años (alrededor de 5 meses) en train y 0.17 años (casi 2 meses) en test_modelo.
La edad más avanzada fue de 80 años y 76 años, respectivamente.
Esto refleja una gran diversidad en el rango de edades de los pasajeros, desde bebés hasta adultos mayores.

**(f) Pasajeros por puerto de embarque**

Los pasajeros embarcaron principalmente en tres puertos:

* Southampton (S): la mayoría de los pasajeros.

* Cherbourg (C): principalmente viajeros de primera clase.

* Queenstown (Q): un grupo reducido, en su mayoría de tercera clase.

Estos valores se pueden confirmar con la función contar_pasajeros_por_puerto_embarque(), que muestra el conteo exacto por cada código de puerto.

**(g) Pasajeros que viajaron solos o con familiares**

En el conjunto train, 537 pasajeros viajaban solos y 354 con familia.
En test_modelo, 255 viajaban solos y 165 acompañados.
Esto evidencia que, en ambos conjuntos, los pasajeros que viajaban sin familiares fueron mayoría, lo cual coincide con las características socioeconómicas del viaje (muchos eran hombres adultos que viajaban solos).

**Punto 9**


In [20]:
print("aca va el punto 9")

aca va el punto 9


**Punto 10**

In [22]:

train = train.copy()
train.loc[:, "Familiares"] = train["SibSp"].fillna(0) + train["Parch"].fillna(0)

def clasificar(n):
    if pd.isna(n) or n == 0:
        return "Grupo 1: sin familiares"
    elif 1 <= n <= 3:
        return "Grupo 2: familia pequeña"
    else:
        return "Grupo 3: familia grande"

train.loc[:, "Grupo_Familiar"] = train["Familiares"].apply(clasificar)
orden = ["Grupo 1: sin familiares", "Grupo 2: familia pequeña", "Grupo 3: familia grande"]
train.loc[:, "Grupo_Familiar"] = pd.Categorical(train["Grupo_Familiar"], categories=orden, ordered=True)

display(train["Grupo_Familiar"].value_counts().sort_index())

dist_cabinas = (
    train.assign(Cabin_cat=train["Cabin"].fillna("Sin registro"))
         .groupby("Grupo_Familiar")["Cabin_cat"]
         .value_counts()
         .rename("conteo")
)
display(dist_cabinas)

tabla_cabinas = (
    train.assign(Cabin_cat=train["Cabin"].fillna("Sin registro"))
         .pivot_table(index="Grupo_Familiar", columns="Cabin_cat", aggfunc="size", fill_value=0)
         .sort_index()
)
display(tabla_cabinas)


Grupo_Familiar
Grupo 1: sin familiares     537
Grupo 2: familia pequeña    292
Grupo 3: familia grande      62
Name: count, dtype: int64

Grupo_Familiar            Cabin_cat      
Grupo 1: sin familiares   Sin registro       443
                          D                    3
                          E101                 3
                          F33                  3
                          B28                  2
                                            ... 
Grupo 2: familia pequeña  E68                  1
                          F E69                1
Grupo 3: familia grande   Sin registro        56
                          C23 C25 C27          4
                          B57 B59 B63 B66      2
Name: conteo, Length: 153, dtype: int64

Cabin_cat,A10,A14,A16,A19,A20,A23,A24,A26,A31,A32,...,F E69,F G63,F G73,F2,F33,F38,F4,G6,Sin registro,T
Grupo_Familiar,,,,,,,,,,,,,,,,,,,,,
Grupo 1: sin familiares,1,1,0,1,0,1,1,1,1,1,...,0,1,2,0,3,1,0,0,443,1
Grupo 2: familia pequeña,0,0,1,0,1,0,0,0,0,0,...,1,0,0,3,0,0,2,4,188,0
Grupo 3: familia grande,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,56,0
